## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-05-13-08-30 +0000,nypost,Trump threatens to ratchet up tariffs on EU if...,https://nypost.com/2025/08/05/us-news/trump-th...
1,2025-08-05-13-05-11 +0000,bbc,Texas Republicans vote to arrest Democrats blo...,https://www.bbc.com/news/articles/c93dkwnx94ro...
2,2025-08-05-13-04-00 +0000,wsj,Opinion | The Defenestration of the BLS Commis...,https://www.wsj.com/opinion/the-defenestration...
3,2025-08-05-13-03-00 +0000,wsj,Opinion | There’s No Shortage of Sugar on the ...,https://www.wsj.com/opinion/theres-no-shortage...
4,2025-08-05-13-02-27 +0000,nyt,"Trump Questions Discrimination Claims, Even On...",https://www.nytimes.com/2025/08/05/business/cf...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2397/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,51
179,new,22
75,gaza,18
324,will,12
3,up,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...,121
116,2025-08-05-01-35-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...,103
255,2025-08-04-16-50-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...,94
127,2025-08-05-01-00-00 +0000,wsj,President Trump’s high-stakes push to create m...,https://www.wsj.com/politics/texas-redistricti...,91
203,2025-08-04-20-30-20 +0000,nypost,Ex-Obama officials face federal grand jury pro...,https://nypost.com/2025/08/04/us-news/ex-obama...,91


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,121,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...
174,57,2025-08-04-21-48-14 +0000,nypost,Brazil’s Supreme Court orders house arrest for...,https://nypost.com/2025/08/04/world-news/brazi...
59,54,2025-08-05-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Israel’s new plan f...,https://nypost.com/2025/08/05/us-news/ny-postc...
116,52,2025-08-05-01-35-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...
275,49,2025-08-04-15-16-00 +0000,wsj,Elon Musk Gets $23.7 Billion Stock Award From ...,https://www.wsj.com/business/autos/tesla-appro...
183,48,2025-08-04-21-20-09 +0000,nypost,NYC’s iconic Flatiron Building will get bathed...,https://nypost.com/2025/08/04/business/nycs-ic...
189,47,2025-08-04-21-04-00 +0000,wsj,The State Department may require travelers ent...,https://www.wsj.com/politics/policy/trump-admi...
40,40,2025-08-05-11-32-17 +0000,nypost,Child services worker arrested after boy dies ...,https://nypost.com/2025/08/05/us-news/child-se...
125,40,2025-08-05-01-03-45 +0000,nypost,Clip of Sydney Sweeney’s expert shooting range...,https://nypost.com/2025/08/04/us-news/clip-of-...
129,40,2025-08-05-00-50-22 +0000,nypost,Oil baron CEO claims he was subject of ‘sham’ ...,https://nypost.com/2025/08/04/us-news/oil-baro...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
